### 패키지 선언

In [4]:
import torch
import torch.nn as nn
import torchvision.datasets as dataset
import torchvision.transforms as transform
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

## Dataset 다운로드

In [5]:
# Training dataset 다운로드
cifar10_train = dataset.CIFAR10(root = "./",
                            train = True,
                            transform = transform.ToTensor(),
                            download = True)
# Testing dataset 다운로드
cifar10_test = dataset.CIFAR10(root = "./",
                            train = False,
                            transform = transform.ToTensor(),
                            download = True)

dataLabel = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

Files already downloaded and verified
Files already downloaded and verified


## LeNet5 모델 정의
- Fully Connected Layer -> nn.Linear(in_features, out_features)
- Convolutional Layer -> nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
- ReLU -> nn.ReLU()
- Max Pooling -> nn.MaxPool2d(kernel_size, stride)


In [ ]:
class LeNet5(nn.Module):

  def __init__(self):
    super(LeNet5, self).__init__()
    # 신경망 파라미터 초기화 (Conv 2개, FC 3개, ReLU, MaxPool)
    self.conv1 = nn.Conv2d(3, 6, 5, 1, 0)
    self.relu = nn.ReLU()
    self.max_pool = nn.MaxPool2d(2, 2)

    self.conv2 = nn.Conv2d(6, 16, 5, 1, 0)

    self.fc1 = nn.Linear(400, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    #-----------------------------------------------------------------
    # Convolution, MaxPool layer 추가 필요 (발표 자료 중 1~6번)
    y = self.conv1(x)
    y = self.relu(y)
    y = self.max_pool(y)

    y = self.conv2(y)
    y = self.relu(y)
    y = self.max_pool(y)
    #-----------------------------------------------------------------

    y = y.view(-1, 400) # feature map 평탄화

    #-----------------------------------------------------------------
    # Fully Connected layer 추가 필요 (발표 자료 중 8~12번)
    y = self.fc1(y)
    y = self.relu(y)
    y = self.fc2(y)
    y = self.relu(y)
    y = self.fc3(y)
    #-----------------------------------------------------------------

    return y


In [3]:
class LeNet5(nn.Module):

  def __init__(self):
    super(LeNet5, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, stride = 1, padding = 1)

    self.relu = nn.ReLU()
    self.avg_pool = nn.AvgPool2d(2, 2)

    self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1)

    self.conv3 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1)

    self.conv4 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, stride = 1, padding = 1)

    self.conv5 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1)

    self.conv6 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, stride = 1, padding = 1)

    self.fc1 = nn.Linear(8192, 256)

    self.fc2 = nn.Linear(256, 256)

    self.fc3 = nn.Linear(256, 10)

  def forward(self, x):
    y = self.conv1(x)
    y = self.relu(y)
    y = self.conv2(y)
    y - self.relu(y)
    y = self.avg_pool(y)

    y = self.conv3(y)
    y = self.relu(y)
    y = self.conv4(y)
    y = self.relu(y)
    y = self.avg_pool(y)

    y = self.conv5(y)
    y = self.relu(y)
    y = self.conv6(y)
    y = self.relu(y)
    y = self.avg_pool(y)

    y = y.view(-1, 8192) # feature map 평탄화

    y = self.fc1(y)
    y = self.relu(y)
    y = self.fc2(y)
    y = self.relu(y)
    y = self.fc3(y)

    return y

NameError: ignored

## Hyper-parameters 지정

In [4]:
batch_size = 100
learning_rate = 0.1
training_epochs = 15
loss_function = nn.CrossEntropyLoss()
network = LeNet5()
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate)

data_loader = DataLoader(dataset=cifar10_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

## CNN 학습을 위한 반복문 선언

In [5]:
network.train()
network = network.to('cuda')
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(data_loader)

  for img, label in data_loader:
    img = img.to('cuda')
    label = label.to('cuda')
    pred = network(img)

    loss = loss_function(pred, label)
    optimizer.zero_grad() # gradient 초기화
    loss.backward()
    optimizer.step()

    avg_cost += loss / total_batch

  print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')

Epoch: 1 Loss = 2.302663
Epoch: 2 Loss = 2.297198
Epoch: 3 Loss = 2.273635
Epoch: 4 Loss = 2.028105
Epoch: 5 Loss = 1.857234
Epoch: 6 Loss = 1.733637
Epoch: 7 Loss = 1.550548
Epoch: 8 Loss = 1.384016
Epoch: 9 Loss = 1.253887
Epoch: 10 Loss = 1.160581
Epoch: 11 Loss = 1.066038
Epoch: 12 Loss = 0.969038
Epoch: 13 Loss = 0.871472
Epoch: 14 Loss = 0.765190
Epoch: 15 Loss = 0.655425
Learning finished


## 학습이 완료된 모델을 이용해 정답률 확인

In [2]:
network.eval()
network = network.to('cpu')
img_test = torch.tensor(np.transpose(cifar10_test.data,(0,3,1,2))) / 255.
label_test = torch.tensor(cifar10_test.targets)

with torch.no_grad(): # test에서는 기울기 계산 제외
  prediction = network(img_test) # 전체 test data를 한번에 계산

  correct_prediction = torch.argmax(prediction, 1) == label_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

NameError: ignored

## 예측 결과 확인

In [6]:
dataIdx = 1919
tmp_img = img_test[dataIdx]
tmp_label = label_test[dataIdx]

with torch.no_grad():
  prediction = network(tmp_img)

prediction_num = torch.argmax(prediction, 1)

print(prediction)
print(dataLabel[prediction_num])

plt.imshow(np.transpose(tmp_img,(1,2,0)))
plt.show

NameError: ignored